# Automated ML

In [ ]:
# Imports

import os
import joblib
import numpy as np
import pandas as pd
import requests
import json

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.core import Model, InferenceConfig
from azureml.core.webservice import AciWebservice

## Dataset

### Overview


In [ ]:
# Worspace and experiment

ws = Workspace.from_config()
experiment_name = 'automl-experiment'
project_folder = './automl-project'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
# Create compute cluster

compute_cluster_name = "alpha"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, please use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

In [ ]:
# Dataset

found = False
key = "Heart Failure Dataset"
description_text = "Heart Failure Dataset for Udacity Project 3"

if key in ws.datasets.keys():
        found = True
        dataset = ws.datasets[key]

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/peppegili/3_Capstone_Project_ML_Engineer/blob/master/data/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

## AutoML Configuration

In [ ]:
# AutoML config

automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "n_cross_validations": 4,
    "primary_metric" : 'accuracy'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task="classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path=project_folder,
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log="automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# Submit the experiment
automl = experiment.submit(automl_config)

## Run Details

In [ ]:
RunDetails(automl).show()
automl.wait_for_completion(show_output=True)

assert(automl.get_status() == "Completed")

## Best Model

In [ ]:
best_run_automl, best_model_automl = automl.get_output()
best_run_metrics_automl = best_run_automl.get_metrics()

print("\n")
print('Best run ID: ', best_run_automl.id)
print('Best run Accuracy: ', best_run_metrics_automl['accuracy'])
print('Metrics: ', best_run_metrics_automl)
print("\n")

In [ ]:
# Save the best model
joblib.dump(best_model_automl, filename='outputs/best_model_automl.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

Register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Register the best model
#best_run_automl.upload_file('outputs/best_model_automl.pkl', 'outputs/best_model_automl.pkl')
model = best_run_automl.register_model(model_name='best_model_automl', model_path='outputs/best_model_automl.pkl')

# Deploy the model
service_name = 'automl-deploy'
inference_config = InferenceConfig(entry_script='score.py', environment=best_run_automl.get_environment())
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                           memory_gb=1,
                                                           auth_enabled=True,
                                                           enable_app_insights=True,
                                                           description='AutoML model deploy')

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_deployment_config,
                       overwrite=True
                      )

service.wait_for_deployment(show_output=True)


print('Deployment state: ', service.state)
print('Scoring URI: ', service.scoring_uri)

Send a request to the web service you deployed to test it.

In [ ]:
# Send a POST requests to the web service

# URL for the web service
scoring_uri = str(service.scoring_ur)
# If the service is authenticated, set the key or token
primary, secondary = service.get_keys()
key = str(primary)

# Two sets of data to score, so we get two results back
data = {"data":
        [
            {
                'age': 50, 
                'anaemia': 1, 
                'creatinine_phosphokinase': 230,
                'diabetes': 0,
                'ejection_fraction': 38,
                'high_blood_pressure': 1,
                'platelets': 390000,
                'serum_creatinine': 1.8,
                'serum_sodium': 135,
                'sex': 1.0,
                'smoking': 0,
                'time': 14
            }
        ]
       }

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

Print the logs of the web service and delete the service

In [ ]:
# Logs of the web service
print(service.get_logs())

# Delete the service
service.delete()